In [46]:
import sys
import pandas as pd
import torch
import sklearn
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from transformers import (
    DistilBertTokenizer,
    DistilBertModel,
    Trainer,
    TrainingArguments,
)
from transformers import AutoTokenizer, AutoModel
from transformers import DistilBertModel
from sklearn.metrics import accuracy_score
from torch.utils.data import DataLoader, TensorDataset
from sklearn.metrics import (
    confusion_matrix,
    accuracy_score,
    precision_score,
    recall_score,
    f1_score,
)
from sklearn.metrics import classification_report
from sklearn.model_selection import StratifiedKFold
from typing import List
import timeit
import joblib
from sklearn.utils import shuffle
from sentence_transformers import SentenceTransformer, util
from sklearn.utils import shuffle
from torch.utils.data import DataLoader, SequentialSampler
import gc
import warnings
import os
from IPython.display import clear_output
import warnings

warnings.filterwarnings(
    "ignore",
    category=FutureWarning,
)
pd.options.mode.chained_assignment = None

In [29]:
%pip install Bachelor_arbeit_modul
import Bachelor_arbeit_modul as bam
clear_output()

In [31]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

## Training & Test

In [68]:
final_train_street = pd.read_parquet(
    "./src/Bachelor_arbeit_modul/Data/Street/street_train_filtered.parquet"
)
final_test_street = pd.read_parquet(
    "./src/Bachelor_arbeit_modul/Data/Street/street_test_filtered.parquet"
)
df_street = shuffle(pd.concat([final_train_street, final_test_street]), random_state=42)

final_test_street

,STREET_DF1,STREET_DF2,STREET_WORDDIFF_LOCALITY,BEMERKUNG,LABELS_STREET,names
5343,Im Trutz Frankfurt,Im Trutz,0.980000,semi_identisch,Match,Im Trutz Frankfurt Im Trutz
1724,An der Riehe,An der Riehe,1.000000,identisch,Match,An der Riehe An der Riehe
1401,Henkelstraße,Henkelstr.,0.980000,semi_identisch,Match,Henkelstraße Henkelstr.
4638,Neusser Straße,Neusser str.,0.980000,semi_identisch,Match,Neusser Straße Neusser str.
1020,Gerresheimer Landstraße,Gerresheimer Landstr.,0.980000,semi_identisch,Match,Gerresheimer Landstraße Gerresheimer Landstr.
...,...,...,...,...,...,...
4981,Hunteburger Straße,Oranienburger Str.,0.648649,nicht_identisch,Not Match,Hunteburger Straße Oranienburger Str.
1950,Gutenbergstraße,Rotenbergstr.,0.689655,nicht_identisch,Not Match,Gutenbergstraße Rotenbergstr.
1253,Am Glaswerk,Am Kaiserkai,0.666667,nicht_identisch,Not Match,Am Glaswerk Am Kaiserkai
8117,Wohnbacher Straße,Schwabacher Str.,0.647059,nicht_identisch,Not Match,Wohnbacher Straße Schwabacher Str.


In [ ]:
tokenizer = DistilBertTokenizer.from_pretrained("distilbert-base-german-cased")
street_train_serialized = bam.Fine_Tuning.serialization(
    final_train_street, street_ser=True, company_ser=False, label=True
)
street_train_dataset = bam.Fine_Tuning.tokenization(
    street_train_serialized,
    tokenizer,
    tok_street=True,
    tok_company=False,
    label=True,
)
street_train_dataset

In [ ]:
street_test_serialized = bam.Fine_Tuning.serialization(
    final_test_street, street_ser=True, company_ser=False, label=True
)
street_test_dataset = bam.Fine_Tuning.tokenization(
    street_test_serialized,
    tokenizer,
    tok_street=True,
    tok_company=False,
    label=True,
)
street_test_dataset

In [35]:
full_street = bam.Fine_Tuning.train_and_evaluate(
    device,
    street_train_dataset,
    street_test_dataset,
    final_test_street,
    dropout=0.1,
    without_activation_function=True,
    optimizer_name="adamw",
    output_dir="./src/Bachelor_arbeit_modul/Data/Street/model",
    num_train_epochs=3,
    per_device_train_batch_size=32,
    learning_rate = 5e-5,
    logging_dir="./src/Bachelor_arbeit_modul/Data/Street/model",
    train_street=True,
    train_company=False,
    weight_decay = 0.02
)
full_street

Some weights of DistilBertForCustomClassification were not initialized from the model checkpoint at distilbert-base-german-cased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Step,Training Loss,Validation Loss,Accuracy
100,0.156900,0.081263,0.976562
200,0.078700,0.062589,0.984003
300,0.070700,0.064234,0.984003
400,0.043700,0.107542,0.975446
500,0.048600,0.073272,0.982143
600,0.043500,0.062620,0.984375
700,0.023700,0.073773,0.981771
800,0.018100,0.080293,0.982143
900,0.013600,0.074243,0.984375
1000,0.029200,0.068270,0.984747


Total training steps: 1008
Final training loss: 0.052403311688630355
Training metrics:
train_runtime: 125.9129
train_samples_per_second: 256.106
train_steps_per_second: 8.006
total_flos: 241950410018244.0
train_loss: 0.052403311688630355
epoch: 3.0


Evaluation metrics:
eval_loss: 0.06258927285671234
eval_accuracy: 0.9840029761904762
eval_runtime: 3.9648
eval_samples_per_second: 677.973
eval_steps_per_second: 84.747
epoch: 3.0
Confusion Matrix: 
[[1342   16]
 [  27 1303]]
accuracy : 0.9840029761904762
precision : 0.9878695981804397
recall : 0.9796992481203007
f1 : 0.9837674594186485
specificity : 0.9882179675994109
None


,STREET_DF1,STREET_DF2,STREET_WORDDIFF_LOCALITY,BEMERKUNG,LABELS_STREET,names,STREET_SERIALIZED,PREDICTIONS_STREET,TRUE_LABELS_STREET
5343,Im Trutz Frankfurt,Im Trutz,0.980000,semi_identisch,1,Im Trutz Frankfurt Im Trutz,Im Trutz Frankfurt [SEP] Im Trutz,1,1
1724,An der Riehe,An der Riehe,1.000000,identisch,1,An der Riehe An der Riehe,An der Riehe [SEP] An der Riehe,1,1
1401,Henkelstraße,Henkelstr.,0.980000,semi_identisch,1,Henkelstraße Henkelstr.,Henkelstraße [SEP] Henkelstr.,1,1
4638,Neusser Straße,Neusser str.,0.980000,semi_identisch,1,Neusser Straße Neusser str.,Neusser Straße [SEP] Neusser str.,1,1
1020,Gerresheimer Landstraße,Gerresheimer Landstr.,0.980000,semi_identisch,1,Gerresheimer Landstraße Gerresheimer Landstr.,Gerresheimer Landstraße [SEP] Gerresheimer Lan...,1,1
...,...,...,...,...,...,...,...,...,...
4981,Hunteburger Straße,Oranienburger Str.,0.648649,nicht_identisch,0,Hunteburger Straße Oranienburger Str.,Hunteburger Straße [SEP] Oranienburger Str.,0,0
1950,Gutenbergstraße,Rotenbergstr.,0.689655,nicht_identisch,0,Gutenbergstraße Rotenbergstr.,Gutenbergstraße [SEP] Rotenbergstr.,0,0
1253,Am Glaswerk,Am Kaiserkai,0.666667,nicht_identisch,0,Am Glaswerk Am Kaiserkai,Am Glaswerk [SEP] Am Kaiserkai,0,0
8117,Wohnbacher Straße,Schwabacher Str.,0.647059,nicht_identisch,0,Wohnbacher Straße Schwabacher Str.,Wohnbacher Straße [SEP] Schwabacher Str.,0,0


## Company

In [67]:
final_train_company = pd.read_parquet(
    "./src/Bachelor_arbeit_modul/Data/Company/company_train_14_12.parquet"
)

final_train_company.reset_index(drop=True, inplace=True)


final_test_company = pd.read_parquet(
    "./src/Bachelor_arbeit_modul/Data/Company/company_test_14_12.parquet"
)
final_test_company.reset_index(drop=True, inplace=True)
df_company = shuffle(
    pd.concat([final_train_company, final_test_company]), random_state=42
)
df_company.drop_duplicates(subset={"COMPANYNAME_DF1", "COMPANYNAME_DF2"}, inplace=True)

final_test_company

,COMPANYNAME_DF1,COMPANYNAME_DF2,COMPANYNAME_WORDDIFF,LABELS_COMPANY,BEMERKUNG
0,Christi Himmelfahrt,RS Print & Promotion GmbH Werbemittel und Druc...,0.273973,Not Match,nicht identisch
1,triathlon.de,triathlon.de GmbH,0.990000,Match,semi_identisch
2,Kita Kleine Strolche,Kita Kleine Strolche gGmbH,0.990000,Match,semi_identisch
3,Dennree GmbH,Henning Larsen GmbH,0.645161,Not Match,nicht identisch
4,Emil Baltzer GmbH & Co. KG,Finance Advisory GmbH & Co. KG,0.642857,Not Match,nicht identisch
...,...,...,...,...,...
2611,Feldmann Transporte,Feldmann Transporte UG (haftungsbeschränkt),0.990000,Match,semi_identisch
2612,team baucenter,NBC Private Markets GmbH,0.315789,Not Match,nicht identisch
2613,Marlin,Marlink GmbH,0.990000,Match,semi_identisch
2614,Kapsmedia GmbH,Kapsmedia GmbH,1.000000,Match,identisch


In [ ]:
tokenizer = DistilBertTokenizer.from_pretrained("distilbert-base-german-cased")
company_train_serialized = bam.Fine_Tuning.serialization(
    final_train_company, street_ser=False, company_ser=True, label=True
)
company_train_dataset = bam.Fine_Tuning.tokenization(
    company_train_serialized,
    tokenizer,
    tok_street=False,
    tok_company=True,
    label=True,
)
company_train_dataset

In [ ]:
tokenizer = DistilBertTokenizer.from_pretrained("distilbert-base-german-cased")
company_test_serialized = bam.Fine_Tuning.serialization(
    final_test_company, street_ser=False, company_ser=True, label=True
)
company_test_dataset = bam.Fine_Tuning.tokenization(
    company_test_serialized,
    tokenizer,
    tok_street=False,
    tok_company=True,
    label=True,
)
company_test_dataset

In [43]:
full_company = bam.Fine_Tuning.train_and_evaluate(
    device,
    company_train_dataset,
    company_test_dataset,
    final_test_company,
    dropout=0.1,
    #activation_position='after',
    without_activation_function=True,
    optimizer_name="adamw",
    output_dir="./src/Bachelor_arbeit_modul/Data/Company/model",
    num_train_epochs=5,
    per_device_train_batch_size=32,
    learning_rate=5e-5,
    logging_dir="./src/Bachelor_arbeit_modul/Data/Company/model",
    train_street=False,
    train_company=True,
    weight_decay = 0.02
)
full_company

Some weights of DistilBertForCustomClassification were not initialized from the model checkpoint at distilbert-base-german-cased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Step,Training Loss,Validation Loss,Accuracy
100,0.119800,0.036895,0.991590
200,0.044300,0.036349,0.991208
300,0.014300,0.034544,0.993502
400,0.005600,0.036808,0.993119
500,0.007900,0.043581,0.992355
600,0.004400,0.031725,0.993119
700,0.001100,0.036917,0.993119
800,0.003500,0.045784,0.993502
900,0.002600,0.041805,0.992355
1000,0.001900,0.041771,0.992737


Total training steps: 1100
Final training loss: 0.018701874813572928
Training metrics:
train_runtime: 178.7693
train_samples_per_second: 196.902
train_steps_per_second: 6.153
total_flos: 573749420428800.0
train_loss: 0.018701874813572928
epoch: 5.0


Evaluation metrics:
eval_loss: 0.031724777072668076
eval_accuracy: 0.9931192660550459
eval_runtime: 3.9455
eval_samples_per_second: 663.038
eval_steps_per_second: 82.88
epoch: 5.0
Confusion Matrix: 
[[1155   12]
 [   6 1443]]
accuracy : 0.9931192660550459
precision : 0.9917525773195877
recall : 0.9958592132505176
f1 : 0.993801652892562
specificity : 0.9897172236503856
None


NameError: name 'll' is not defined

In [44]:
full_company

,COMPANYNAME_DF1,COMPANYNAME_DF2,COMPANYNAME_WORDDIFF,LABELS_COMPANY,BEMERKUNG,COMPANY_SERIALIZED,PREDICTIONS_COMPANY,TRUE_LABELS_COMPANY
0,Christi Himmelfahrt,RS Print & Promotion GmbH Werbemittel und Druc...,0.273973,0,nicht identisch,Christi Himmelfahrt [SEP] RS Print & Promotion...,0,0
1,triathlon.de,triathlon.de GmbH,0.990000,1,semi_identisch,triathlon.de [SEP] triathlon.de GmbH,1,1
2,Kita Kleine Strolche,Kita Kleine Strolche gGmbH,0.990000,1,semi_identisch,Kita Kleine Strolche [SEP] Kita Kleine Strolch...,1,1
3,Dennree GmbH,Henning Larsen GmbH,0.645161,0,nicht identisch,Dennree GmbH [SEP] Henning Larsen GmbH,0,0
4,Emil Baltzer GmbH & Co. KG,Finance Advisory GmbH & Co. KG,0.642857,0,nicht identisch,Emil Baltzer GmbH & Co. KG [SEP] Finance Advis...,0,0
...,...,...,...,...,...,...,...,...
2611,Feldmann Transporte,Feldmann Transporte UG (haftungsbeschränkt),0.990000,1,semi_identisch,Feldmann Transporte [SEP] Feldmann Transporte ...,1,1
2612,team baucenter,NBC Private Markets GmbH,0.315789,0,nicht identisch,team baucenter [SEP] NBC Private Markets GmbH,0,0
2613,Marlin,Marlink GmbH,0.990000,1,semi_identisch,Marlin [SEP] Marlink GmbH,1,1
2614,Kapsmedia GmbH,Kapsmedia GmbH,1.000000,1,identisch,Kapsmedia GmbH [SEP] Kapsmedia GmbH,1,1


# Matching Process

In [47]:
df_hugging = pd.read_parquet(
    "./src/Bachelor_arbeit_modul/Data/hugging_face_clean.parquet"
)

df_hugging

,COMPANYNAME,STREET,POSTALCODE,CITY,HSNR
2,Bruno Söhnle GmbH Uhrenatelier Glashütte/Sa.,Dresdner Str.,01768,Glashütte,14
3,Funke & Will AG,Zum Fliegerhorst,01558,Großenhain,11
4,f6 Cigarettenfabrik GmbH & Co. KG,Junghansstr.,01277,Dresden,5
5,BASF Schwarzheide GmbH,Schipkauer Str.,01987,Schwarzheide,1
6,Elbe Flugzeugwerke GmbH,Grenzstr.,01109,Dresden,1
...,...,...,...,...,...
261867,gert feistkorn,Haunoldshofener Weg,90599,Dietenhofen,3
261868,christian hans,Schweppermannstr.,90408,Nürnberg,23
261869,peter fürderer,An der Fleischbrücke,90455,Nürnberg,1-3
261870,peter kühnl,Habsburgerstr.,90475,Nürnberg,24


In [48]:
df_osm = pd.read_parquet(
    "./src/Bachelor_arbeit_modul/Data/df_osm_all.parquet"
)
df_osm = df_osm[["COMPANYNAME", "CITY", "POSTALCODE", "STREET", "HSNR"]]
df_osm

,COMPANYNAME,CITY,POSTALCODE,STREET,HSNR
0,BBBank,Weinheim,69469,Dürrestraße,1
1,Arbeitsgericht Karlsruhe,Karlsruhe,76133,Ritterstraße,12
2,Sparkasse,Karlsruhe,76199,Tulpenstraße,4
3,Marienapotheke,Karlsruhe,76137,Marienstraße,43
4,Spitzwegapotheke,Karlsruhe,76137,Südendstraße,29
...,...,...,...,...,...
401316,Gemeinde Herbsleben,Herbsleben,99955,Mitteltor,4
401317,Johanniter-Unfall-Hilfe e.V.,Eisenberg,07607,Klosterlausnitzer Straße,23
401318,Gemeindeverwaltung,Frankenblick,96528,Alter Weg,24
401319,Anja Henkel,Suhl,98527,Albert-Schweitzer-Straße,1


In [50]:
matched= bam.Matcher.matcher(
    df_osm,
    df_hugging,
    batch_size=32,
    street_model_path = "./src/Bachelor_arbeit_modul/Data/Street/model/model",
    company_model_path="./src/Bachelor_arbeit_modul/Data/Company/model/model",
)
matched

,COMPANYNAME_DF1,POSTALCODE_DF1,CITY_DF1,STREET_DF1,HSNR_DF1,INDEX_SOURCE_DF1,INDEX_SOURCE_DF2,COMPANYNAME_DF2,POSTALCODE_DF2,CITY_DF2,STREET_DF2,HSNR_DF2,HSNR_SIM,PREDICTIONS_STREET,PREDICTIONS_COMPANY,MATCH_ID
692,Dresden International School,01067,Dresden,Annenstraße,9,98090,133909,DIS Dresden International School gemeinnützige...,01067,Dresden,Annenstr.,9,1.0,1,1,1
2967,winkler.digital GmbH,01067,Dresden,Kraftwerk Mitte,7,95268,134691,winkler.digital UG (haftungsbeschränkt),01067,Dresden,Kraftwerk Mitte,7,1.0,1,1,2
3007,Xilinx Dresden GmbH,01067,Dresden,Kramergasse,4,93944,134407,Xilinx Dresden GmbH,01067,Dresden,Kramergasse,4,1.0,1,1,3
3021,eureos GmbH,01067,Dresden,Kramergasse,4,94196,139081,eureos corporate finance gmbh,01067,Dresden,Kramergasse,4,1.0,1,1,4
3022,eureos GmbH,01067,Dresden,Kramergasse,4,94196,142437,eureos gmbh steuerberatungsgesellschaft rechts...,01067,Dresden,Kramergasse,4,1.0,1,1,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1216028,Apotheke am Europakanal,90768,Fürth,Kurt-Scherzer-Straße,4,28527,345074,apotheke am europakanal,90768,Fürth,Kurt-Scherzer-Str.,4,1.0,1,1,1914
1216593,Frosch-Apotheke,90768,Fürth,Vacher Straße,462,17145,334619,Frosch - Apotheke e. K.,90768,Fürth,Vacher Str.,462,1.0,1,1,1915
1217995,SF Gebäudereinigung,95030,Hof,Hofer Straße,21,27772,332126,SF Gebäudereinigung GmbH,95030,Hof,Hofer Str.,21,1.0,1,1,1916
1218666,CITYCOM GmbH,97239,Aub,Industriestraße,5-9,16236,294641,CITYCOM GmbH,97239,Aub,Industriestr.,5-9,1.0,1,1,1917


In [53]:
matched["MATCH_ID"].nunique()

1918

In [65]:
matched[matched["STREET_DF1"] != matched["STREET_DF2"]][["STREET_DF1", "STREET_DF2"]]

,STREET_DF1,STREET_DF2
692,Annenstraße,Annenstr.
3172,Magdeburger Straße,Magdeburger Str.
3308,Maternistraße,Maternistr.
4552,Schloßstraße,Schloßstr.
4827,Schweriner Straße,Schweriner Str.
...,...,...
1216028,Kurt-Scherzer-Straße,Kurt-Scherzer-Str.
1216593,Vacher Straße,Vacher Str.
1217995,Hofer Straße,Hofer Str.
1218666,Industriestraße,Industriestr.


In [71]:
matched[matched["COMPANYNAME_DF1"] != matched["COMPANYNAME_DF2"]][["COMPANYNAME_DF1", "COMPANYNAME_DF2"]].sample(10)

,COMPANYNAME_DF1,COMPANYNAME_DF2
432390,Neu Seeschifffahrt,Neu Seeschiffahrt GmbH
89964,Bethesda Apotheke,bethesda-apotheke
766033,Kunsthochschule für Medien,Media Initiative Ghana e.V. Verein zur Förderu...
1133233,Cuntz Autotechnik,Cuntz GmbH
890610,seracom,Seracom Documentations GmbH
570669,Bismarck-Apotheke,Bismarck-Apotheke Tatjana Zinnen e.K.
666690,Bären-Apotheke,Bären-Apotheke am Neuenhausplatz e. K.
766792,Kölner Funken Artillerie blau-weiß von 1870 e.V.,Kinder- und Jugendtanzgruppe der Kölner Funken...
750701,Kees Fahrschulen Verwaltungsgesellschaft,Kees Fahrschulen Verwaltungsgesellschaft UG (h...
975271,MotoShop Lanz,Moto Trade Europe GmbH


In [54]:
matched[["COMPANYNAME_DF1", "COMPANYNAME_DF2"]].sample(20)

,COMPANYNAME_DF1,COMPANYNAME_DF2
709798,Image Engineering,Image Engineering Verwaltungs GmbH
1010866,Bain & Company,"Bain & Company Germany, Inc. Zweigniederlassun..."
162219,SWF Sporthotel-Restaurant,"SWF Skisportzentrum, Wanderheim & Sporthotel, ..."
241220,Pflegeagenten GmbH & Co. KG,Pflegeagenten Verwaltungs GmbH
753295,Elektro Engels + Schmitz GmbH,Elektro Engels-Schmitz Gesellschaft mit beschr...
682334,Kreisverwaltung Mettmann,Bildungsakademie für Gesundheits- und Sozialbe...
526219,Brinker Apotheke,Brinker Apotheke Andreas Tidow e.K.
995276,Ludwig Beck,Ludwig Beck Beteiligungs GmbH
702555,Siewert & Kau Computertechnik,Siewert & Kau Service GmbH
799549,Procar Automobile,Procar Automobile GmbH & Co. KG
